# SignalFlow SDK Client Examples

This notebook demonstrates common usage patterns for the SignalFlow SDK.
Use these examples as templates for your own data analysis workflows.

## Table of Contents
1. [Connection & Setup](#1-connection--setup)
2. [Meeting Analysis](#2-meeting-analysis)
3. [Signal Exploration](#3-signal-exploration)
4. [DIKW Knowledge Mining](#4-dikw-knowledge-mining)
5. [Career Insights](#5-career-insights)
6. [Custom Reports](#6-custom-reports)
7. [Data Export](#7-data-export)

## 1. Connection & Setup

In [ ]:
# Install dependencies (run once)
# !pip install pandas matplotlib seaborn

In [ ]:
# Setup path and imports
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import the SDK
from src.app.sdk import SignalFlowClient

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Imports ready")

In [ ]:
# Connect to environment
# Options: 'local', 'staging', 'production'

client = SignalFlowClient(environment="staging")

print(f"📡 Connected to: {client.environment.name}")
print(f"   API URL: {client.environment.api_url}")
print(f"   Supabase: {'✅' if client.supabase else '❌ Not available'}")

In [ ]:
# Health check
health = client.health_check()
if health:
    print(f"✅ API Status: {health.get('status', 'unknown')}")
else:
    print("⚠️ API not responding (may need to start local server)")

## 2. Meeting Analysis

In [ ]:
# Example 2.1: List all meetings
meetings = client.meetings.list(limit=100)
print(f"📅 Found {len(meetings)} meetings")

# Show first 5
for m in meetings[:5]:
    print(f"  • {m.meeting_name[:40]:<40} | Signals: {m.signals_count}")

In [ ]:
# Example 2.2: Convert to DataFrame for analysis
df_meetings = client.meetings.to_dataframe()

print(f"\nDataFrame shape: {df_meetings.shape}")
df_meetings.head()

In [ ]:
# Example 2.3: Filter meetings by date range
from datetime import datetime, timedelta

# Last 7 days
week_ago = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
today = datetime.now().strftime("%Y-%m-%d")

recent_meetings = client.meetings.list(
    date_from=week_ago,
    date_to=today,
    limit=50
)

print(f"📅 Meetings in last 7 days: {len(recent_meetings)}")

In [ ]:
# Example 2.4: Meeting statistics
stats = client.meetings.get_statistics()

print("\n📊 Meeting Statistics")
print("=" * 40)
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Example 2.5: Visualize meeting activity
if len(df_meetings) > 0 and 'created_at' in df_meetings.columns:
    # Group by date
    df_meetings['date'] = pd.to_datetime(df_meetings['created_at']).dt.date
    daily_counts = df_meetings.groupby('date').size()
    
    plt.figure(figsize=(12, 4))
    daily_counts.plot(kind='bar', color='steelblue', alpha=0.7)
    plt.title('Meetings Per Day')
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize")

## 3. Signal Exploration

In [ ]:
# Example 3.1: List all signals
signals = client.signals.list(limit=100)
print(f"📡 Found {len(signals)} signals")

# Show first 5
for s in signals[:5]:
    print(f"  [{s.signal_type:10}] {s.content[:50]}...")

In [ ]:
# Example 3.2: Signal distribution by type
signal_counts = client.signals.get_by_type()

print("\n📊 Signal Distribution")
print("=" * 40)
for sig_type, count in sorted(signal_counts.items(), key=lambda x: -x[1]):
    bar = "█" * min(count, 30)
    print(f"  {sig_type:12} {bar} {count}")

In [ ]:
# Example 3.3: Filter by signal type
# Get only action items
actions = client.signals.list(signal_type="action", limit=20)

print(f"\n📋 Action Items ({len(actions)} found)")
print("=" * 40)
for a in actions[:10]:
    status = "✅" if a.status == "completed" else "⏳"
    print(f"  {status} {a.content[:60]}...")

In [ ]:
# Example 3.4: Semantic search for signals
query = "architecture decisions"  # Change this to search for different topics

results = client.signals.search(query, limit=10)

print(f"\n🔍 Search: '{query}'")
print(f"   Found: {len(results)} signals")
print("=" * 40)
for r in results:
    print(f"  [{r.signal_type}] {r.content[:70]}...")

In [ ]:
# Example 3.5: Signals to DataFrame
df_signals = client.signals.to_dataframe()

print(f"DataFrame shape: {df_signals.shape}")
df_signals.head()

In [ ]:
# Example 3.6: Visualize signal types
if signal_counts:
    plt.figure(figsize=(10, 6))
    colors = ['#4CAF50', '#2196F3', '#FF9800', '#f44336', '#9C27B0', '#607D8B']
    
    types = list(signal_counts.keys())
    counts = list(signal_counts.values())
    
    plt.pie(counts, labels=types, colors=colors[:len(types)], autopct='%1.1f%%', startangle=90)
    plt.title('Signal Distribution by Type')
    plt.axis('equal')
    plt.show()

## 4. DIKW Knowledge Mining

In [ ]:
# Example 4.1: DIKW distribution
dikw_counts = client.dikw.get_by_level()

print("\n📚 DIKW Pyramid")
print("=" * 40)
levels = ['wisdom', 'knowledge', 'information', 'data']
for level in levels:
    count = dikw_counts.get(level, 0)
    bar = "█" * min(count, 25)
    print(f"  {level:12} {bar} {count}")

In [ ]:
# Example 4.2: List knowledge items
knowledge_items = client.dikw.list(level="knowledge", limit=10)

print(f"\n💡 Knowledge Items ({len(knowledge_items)} found)")
print("=" * 40)
for item in knowledge_items:
    title = item.get('title', item.get('content', 'Untitled'))[:50]
    print(f"  • {title}")

In [ ]:
# Example 4.3: DIKW DataFrame
df_dikw = client.dikw.to_dataframe()

if len(df_dikw) > 0:
    print(f"DataFrame shape: {df_dikw.shape}")
    df_dikw.head()
else:
    print("No DIKW data available")

## 5. Career Insights

In [ ]:
# Example 5.1: Recent standups
standups = client.career.get_standups(limit=5)

print(f"\n📝 Recent Standups ({len(standups)} found)")
print("=" * 40)
for s in standups:
    date = s.get('created_at', '')[:10]
    text = s.get('update_text', '')[:60]
    print(f"  [{date}] {text}...")

In [ ]:
# Example 5.2: Career suggestions
suggestions = client.career.get_suggestions(limit=5)

print(f"\n💼 Career Suggestions ({len(suggestions)} found)")
print("=" * 40)
for s in suggestions:
    title = s.get('title', s.get('suggestion', 'Untitled'))[:50]
    print(f"  → {title}")

In [ ]:
# Example 5.3: Skills tracking
df_skills = client.career.to_dataframe("skills")

if len(df_skills) > 0:
    print(f"📊 Skills ({len(df_skills)} tracked)")
    df_skills.head(10)
else:
    print("No skills data available")

## 6. Custom Reports

In [ ]:
# Example 6.1: Weekly summary
summary = client.insights.summarize_week(week_offset=0)

print(f"\n📊 Weekly Summary ({summary.start_date} to {summary.end_date})")
print("=" * 50)
print(f"  Meetings: {summary.meeting_count}")
print(f"  Open Actions: {summary.open_actions}")
print(f"  Completed Actions: {summary.completed_actions}")
print(f"  \n  Signal Breakdown:")
for sig_type, count in summary.signal_counts.items():
    print(f"    • {sig_type}: {count}")
print(f"  \n  Top Themes: {', '.join(summary.top_themes[:5])}")

In [ ]:
# Example 6.2: Trend analysis
trend = client.insights.trend_analysis("meeting_count", days=14)

print(f"\n📈 Meeting Trend ({trend.period})")
print("=" * 40)
print(f"  Direction: {trend.trend_direction}")
print(f"  Change: {trend.change_percent}%")

# Visualize if we have data
if trend.values and len(trend.values) > 1:
    plt.figure(figsize=(10, 4))
    plt.plot(range(len(trend.values)), trend.values, marker='o', linewidth=2)
    plt.title(f'Meeting Trend ({trend.period})')
    plt.xlabel('Days')
    plt.ylabel('Count')
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Example 6.3: Recommendations
recs = client.insights.recommendations()

print("\n💡 Recommendations")
print("=" * 50)
for r in recs:
    priority = {'high': '🔴', 'medium': '🟡', 'low': '🟢'}.get(r['priority'], '⚪')
    print(f"  {priority} [{r['type']}] {r['description']}")

In [ ]:
# Example 6.4: Custom analysis function
def analyze_meeting_productivity(client):
    """Calculate productivity metrics from meeting data."""
    meetings = client.meetings.list(limit=100)
    signals = client.signals.list(limit=500)
    
    # Calculate metrics
    total_meetings = len(meetings)
    meetings_with_signals = len([m for m in meetings if m.signals_count > 0])
    
    actions = [s for s in signals if s.signal_type == 'action']
    completed = [a for a in actions if a.status == 'completed']
    
    return {
        'total_meetings': total_meetings,
        'productive_meetings': meetings_with_signals,
        'productivity_rate': meetings_with_signals / total_meetings if total_meetings else 0,
        'total_actions': len(actions),
        'completed_actions': len(completed),
        'completion_rate': len(completed) / len(actions) if actions else 0,
    }

metrics = analyze_meeting_productivity(client)

print("\n📊 Productivity Metrics")
print("=" * 40)
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.1%}")
    else:
        print(f"  {key}: {value}")

## 7. Data Export

In [ ]:
# Example 7.1: Export to CSV
def export_to_csv(client, output_dir="exports"):
    """Export all data to CSV files."""
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Meetings
    df_meetings = client.meetings.to_dataframe()
    df_meetings.to_csv(f"{output_dir}/meetings.csv", index=False)
    print(f"  ✅ Exported {len(df_meetings)} meetings")
    
    # Signals
    df_signals = client.signals.to_dataframe()
    df_signals.to_csv(f"{output_dir}/signals.csv", index=False)
    print(f"  ✅ Exported {len(df_signals)} signals")
    
    # DIKW
    df_dikw = client.dikw.to_dataframe()
    df_dikw.to_csv(f"{output_dir}/dikw.csv", index=False)
    print(f"  ✅ Exported {len(df_dikw)} DIKW items")
    
    print(f"\n📁 Files saved to {output_dir}/")

# Uncomment to export:
# export_to_csv(client)

In [ ]:
# Example 7.2: Export to JSON
import json

def export_to_json(client, output_file="exports/full_export.json"):
    """Export all data to a single JSON file."""
    import os
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    data = {
        "exported_at": datetime.now().isoformat(),
        "environment": client.environment.name,
        "meetings": [m.__dict__ for m in client.meetings.list(limit=1000)],
        "signals": [s.__dict__ for s in client.signals.list(limit=5000)],
        "dikw": client.dikw.list(limit=1000),
        "stats": {
            "meetings": client.meetings.get_statistics(),
            "signals": client.signals.get_by_type(),
            "dikw": client.dikw.get_by_level(),
        }
    }
    
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2, default=str)
    
    print(f"✅ Exported to {output_file}")

# Uncomment to export:
# export_to_json(client)

## 🎉 Next Steps

Now that you've seen the SDK in action, try:

1. **Modify queries** - Change the filters and limits
2. **Create custom reports** - Combine multiple data sources
3. **Build visualizations** - Use matplotlib/seaborn for charts
4. **Export data** - Save to CSV/JSON for external tools
5. **Automate workflows** - Schedule notebooks with Papermill

See the full SDK documentation in `src/app/sdk/` for more options.